# Kaggle Competition: Child Mind Institute - Detect Sleep States

## Dataset Description
The dataset comprises about 500 multi-day recordings of wrist-worn accelerometer data annotated with two event types: onset, the beginning of sleep, and wakeup, the end of sleep. Your task is to detect the occurrence of these two events in the accelerometer series.

While sleep logbooks remain the gold-standard, when working with accelerometer data we refer to sleep as the longest single period of inactivity while the watch is being worn. For this data, we have guided raters with several concrete instructions:

A single sleep period must be at least 30 minutes in length
A single sleep period can be interrupted by bouts of activity that do not exceed 30 consecutive minutes
No sleep windows can be detected unless the watch is deemed to be worn for the duration (elaborated upon, below)
The longest sleep window during the night is the only one which is recorded
If no valid sleep window is identifiable, neither an onset nor a wakeup event is recorded for that night.
Sleep events do not need to straddle the day-line, and therefore there is no hard rule defining how many may occur within a given period. However, no more than one window should be assigned per night. For example, it is valid for an individual to have a sleep window from 01h00–06h00 and 19h00–23h30 in the same calendar day, though assigned to consecutive nights
There are roughly as many nights recorded for a series as there are 24-hour periods in that series.
Though each series is a continuous recording, there may be periods in the series when the accelerometer device was removed. These period are determined as those where suspiciously little variation in the accelerometer signals occur over an extended period of time, which is unrealistic for typical human participants. Events are not annotated for these periods, and you should attempt to refrain from making event predictions during these periods: an event prediction will be scored as false positive.

Each data series represents this continuous (multi-day/event) recording for a unique experimental subject.

Note that this is a Code Competition, in which the actual test set is hidden. In this public version, we give some sample data in the correct format to help you author your solutions. The full test set contains about 200 series.

**Files and Field Descriptions:**

- <i><u>train_series.parquet</u></i> - Series to be used as training data. Each series is a continuous recording of accelerometer data for a single subject spanning many days.

    - series_id - Unique identifier for each accelerometer series.

    - step - An integer timestep for each observation within a series.

    - timestamp - A corresponding datetime with ISO 8601 format %Y-%m-%dT%H:%M:%S%z.

    - anglez - As calculated and described by the GGIR package, z-angle is a metric derived from individual accelerometer components that is commonly used in sleep detection, and refers to the angle of the arm relative to the vertical axis of the body
    
    - enmo - As calculated and described by the GGIR package, ENMO is the Euclidean Norm Minus One of all accelerometer signals, with negative values rounded to zero. While no standard measure of acceleration exists in this space, this is one of the several commonly computed features
    
    
- <i><u>test_series.parquet</u></i> - Series to be used as the test data, containing the same fields as above. You will predict event occurrences for series in this file.


- <i><u>train_events.csv</u></i> - Sleep logs for series in the training set recording onset and wake events.

    - series_id - Unique identifier for each series of accelerometer data in train_series.parquet.

    - night - An enumeration of potential onset / wakeup event pairs. At most one pair of events can occur for each night.

    - event - The type of event, whether onset or wakeup.

    - step and timestamp - The recorded time of occurence of the event in the accelerometer series.
    

- <u><i>sample_submission.csv</i></u> - A sample submission file in the correct format. See the Evaluation page for more details.

## Challenges

- Create the target variable

# Starting the Project:

In [1]:
# Imports:

# General:
import pandas as pd
import numpy as np

# Data Visualization:
import matplotlib.pyplot as plt
import seaborn as sns

# models:

# Fine Tunning

In [ ]:
# Loading...

train_series = pd.read_parquet('train_series.parquet')
test_series = pd.read_parquet('test_series.parquet')
train_df = pd.read_csv('train_events.csv')
sample = pd.read_csv('sample_submission.csv') 

In [ ]:
train_series.head()

In [ ]:
test_series.head()

In [ ]:
train_df.head()

In [ ]:
sample.head()

## Bringing down memory usage

In [ ]:
# To check the memory usage of the DataFrame:

train_series.info(memory_usage="deep")

### Train_df reduction

In [ ]:
# Select and change data type of columns with int64 to int8
int64 = train_df.select_dtypes(include=['int64']).columns
train_df[int64] = train_df[int64].astype('int8')

# Select and change data type of columns with float64 to float16
float64 = list(train_df.select_dtypes(include=['float64']).columns)
train_df[float64] = train_df[float64].astype('float16')

# Tranforming the column event type from object into boolean
train_df['event'] = train_df['event'].map({'onset': 0, 'wakeup': 1})

# Define the format string to match the datetime format
date_format = "%Y-%m-%dT%H:%M:%S%z"

# Convert 'datetime_str' to datetime using the specified format
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'], format=date_format, utc=True)

# Transgorming the Id column type from object to int8
id_mapping = {unique_id: i for i, unique_id in enumerate(train_df['series_id'].unique())}
train_df['series_id'] = train_df['series_id'].map(id_mapping).astype('int8')

### Train_series reduction

In [ ]:
# To check the memory usage of the DataFrame:

train_series.info(memory_usage="deep")

In [ ]:
# Select and change data type of columns with int64 to int8
int64 = train_series.select_dtypes(include=['int64']).columns
train_series[int64] = train_series[int64].astype('int8')

# Select and change data type of columns with float64 to float16
float64 = list(train_series.select_dtypes(include=['float64']).columns)
train_series[float64] = train_series[float64].astype('float16')

# Select and change data type of columns with float32 to float16
float32 = list(train_series.select_dtypes(include=['float32']).columns)
train_series[float32] = train_series[float32].astype('float16')


# Define the format string to match the datetime format
date_format = "%Y-%m-%dT%H:%M:%S%z"


# Convert 'datetime_str' to datetime using the specified format
train_series['timestamp'] = pd.to_datetime(train_series['timestamp'].head(10000), format=date_format, utc=True)

# Transgorming the Id column type from object to int8
id_mapping = {unique_id: i for i, unique_id in enumerate(train_series['series_id'].unique())}
train_series['series_id'] = train_series['series_id'].map(id_mapping).astype('int8')

In [ ]:
train_series_red = red_mem_usg(train_series[:10000000], series_id=True, timestamp=True)